# ANN Implementation using Pyrenn-LM method

In [ ]:
pip install pyrenn

  Preparing metadata (setup.py) ... done
  Created wheel for pyrenn: filename=pyrenn-0.1-py3-none-any.whl size=9237 sha256=7d3765500821c4882f73e02d93c802094d6feb126015604dd7aa565e707eccec
  Stored in directory: /root/.cache/pip/wheels/88/73/cf/52f87ad9ea9e987087f5c2b03c8d33e837693325a2e0305736
Successfully built pyrenn


In [ ]:
import pandas as pd
import numpy as np
import pyrenn as prn
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('biosensor_data.csv')
dataset.head()

,Glucose,Benzoquinone,T,PH,I
0,4,1.0,20,4,1.441515
1,8,1.0,20,4,2.749113
2,12,1.0,20,4,4.060050
3,16,1.0,20,4,5.096201
4,20,1.0,20,4,5.961336


In [ ]:
X = dataset.iloc[:,0:4]
y = dataset.iloc[:,4]

print(X.head())
print(y.head())

# print(type(X))

   Glucose  Benzoquinone   T  PH
0        4           1.0  20   4
1        8           1.0  20   4
2       12           1.0  20   4
3       16           1.0  20   4
4       20           1.0  20   4
0    1.441515
1    2.749113
2    4.060050
3    5.096201
4    5.961336
Name: I, dtype: float64


In [ ]:
## splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# X_train.shape
X_test.shape

(96, 4)

In [ ]:
# standardizing features

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Compute the Normalized Root Mean Square Error (NRMSE) with leverage adjustments.

def nrmse_leverage(y_true, y_pred, h):
    # h (array-like): Leverage values for each observation.

    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    h = np.array(h)

    # Calculate residuals
    residuals = y_true - y_pred

    # Calculate the numerator (sum of squared leverage-adjusted residuals)
    numerator = np.sum((residuals / (1 - h))**2)

    # Calculate the denominator (total variance of the true values)
    denominator = np.sum((y_true - np.mean(y_true))**2)

    # Calculate the NRMSE
    nrmse = np.sqrt(numerator / denominator)

    return nrmse


In [ ]:
def calc_leverage(X_test):
  perturbation = 1e-5  # Small perturbation value
  leverages = []

  for i in range(X_test.shape[0]):
      # Perturb the i-th input
      X_test_perturbed = X_test.copy()
      X_test_perturbed[i] += perturbation

      # Get predictions with perturbed input
      y_pred_perturbed = prn.NNOut(X_test_perturbed.T, model)

      # Calculate influence (change in predictions)
      leverage = np.abs(y_pred_perturbed - y_pred)
      leverages.append(leverage)
  return leverages;

In [ ]:
# X = X.T
# y = y.T

In [ ]:
y.shape

(320,)

In [ ]:
model = prn.CreateNN([4,4,1])

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [ ]:
type(y_train)

numpy.ndarray

In [ ]:
# Set up cross-validation
n_splits = 10
n_repeats = 2
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# To store results for each fold
nrmse_test_scores = []
r2_scores = []

for repeat in range(n_repeats):
    # print(f'Repeat {repeat + 1}/{n_repeats}')

    for fold, (train_index, test_index) in enumerate(kf.split(X_train)):
        # print(f'  Fold {fold + 1}/{n_splits}')

        # Split the data into train and test sets for this fold
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        # Train the network using Levenberg-Marquardt optimization
        prn.train_LM(X_train_fold.T, y_train_fold.T, model, k_max=100, verbose=1)

        # Predict on the training and test sets
        y_pred = prn.NNOut(X_test_fold.T, model)

        # # Calculate NRMSE for training and test sets
        nrmse_test = nrmse_leverage(y_test_fold, y_pred, calc_leverage(X_test_fold))

        # # Store results for this fold
        nrmse_test_scores.append(nrmse_test)
        r2_scores.append(r2_score(y_test_fold, y_pred))


# Calculate and display the average NRMSE over all repetitions and folds
avg_nrmse_test = np.mean(nrmse_test_scores)
avg_r2_score = np.mean(r2_scores)

print(f'Average Test NRMSE: {avg_nrmse_test:.4f}')
print(f'Average R2 Score: {avg_r2_score:.4f}')

Iteration:  0 		Error:  10.758266624186385 	scale factor:  3.0
Iteration:  1 		Error:  9.359305803953804 	scale factor:  0.3
Iteration:  2 		Error:  8.723440647015122 	scale factor:  0.03
Iteration:  3 		Error:  5.947773539528642 	scale factor:  0.3
Iteration:  4 		Error:  3.942380047067521 	scale factor:  0.03
Iteration:  5 		Error:  2.1145951738093354 	scale factor:  0.03
Iteration:  6 		Error:  1.7217066115483741 	scale factor:  0.03
Iteration:  7 		Error:  1.6123003318002167 	scale factor:  0.03
Iteration:  8 		Error:  1.534032391156216 	scale factor:  0.03
Iteration:  9 		Error:  1.4401576627032937 	scale factor:  0.03
Iteration:  10 		Error:  1.2796776768506886 	scale factor:  0.03
Iteration:  11 		Error:  1.049881213137179 	scale factor:  0.03
Iteration:  12 		Error:  0.7257130936046859 	scale factor:  0.003
Iteration:  13 		Error:  0.5095672343514843 	scale factor:  0.003
Iteration:  14 		Error:  0.4905146531857829 	scale factor:  0.003
Iteration:  15 		Error:  0.39849501815701

In [ ]:
y_pred = prn.NNOut(X_test.T, model)

In [ ]:
r2_score(y_test, y_pred)

0.9674387976711138